In [79]:
import requests
from bs4 import BeautifulSoup
from langchain.prompts import ChatPromptTemplate
from langchain_ollama.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [80]:
web_link='https://www.gamedeveloper.com/design/classic-postmortem-telltale-games-the-walking-dead-2012'

response = requests.get(web_link)
if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')
    text = soup.get_text(separator="\n", strip=True)

In [81]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1000,  
    chunk_overlap=100,  
    length_function=len
)
chunks = text_splitter.split_text(text)


Created a chunk of size 1129, which is longer than the specified 1000


In [82]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")


In [83]:
from langchain_chroma import Chroma


vector_store = Chroma.from_texts(
    texts=chunks,
    collection_name="twd_pm",
    embedding=embeddings,
    persist_directory="./twd",
)

In [85]:
results = vector_store.similarity_search_with_score(
    "The Walking Dead", k=3,
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content}")

* [SIM=0.701780] The Walking Dead
we were mostly able to find the great actors we required in time, but just barely.
The challenge of defining and communicating the essence of our characters in ways that provide talented actors the tools they need in a game context has proven to be a daunting and difficult task. Our actors rarely get to perform with one another—we usually record their parts individually—which makes it hard to offer them context or insights into their character’s motivation. Also, we’re finding that directing the performances during the actual recording sessions is itself a specialized skill set that we need to improve.
Image via Game Developer Magazine, March 2013.
Remaking episode two
While we were focusing intently on episode one, episode two was quietly coming to life in the background on its own.
* [SIM=0.701780] The Walking Dead
we were mostly able to find the great actors we required in time, but just barely.
The challenge of defining and communicating the essenc

In [86]:
retriever = vector_store.as_retriever(
    search_type="similarity",  search_kwargs={"k": 3}
)

In [ ]:
query = "What went wrong with The Walking Dead"
results = retriever.get_relevant_documents(query)  

for i, result in enumerate(results):
    print(f"Resultado {i+1}:")
    print(result.page_content) 
    print()

Resultado 1:
Though we’ve aggressively patched these issues and resolved most of them, you can’t really unring a bell. There are still small content bugs left in the game, and players are too often left with an impression that the game is buggy. All we can do at this point is continue to patch, and take what we’ve learned from this experience to ensure bugs like this don’t happen in the future.
The Walking Devs
For the last eight years, Telltale has been manically committed to the idea that interacting with characters and stories could be some of the best game experiences ever. We’re convinced that stories are not just for cutscenes—stories can be the game themselves.
The Walking Dead

Resultado 2:
Though we’ve aggressively patched these issues and resolved most of them, you can’t really unring a bell. There are still small content bugs left in the game, and players are too often left with an impression that the game is buggy. All we can do at this point is continue to patch, and take 

In [88]:
# Prompt
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

ollama_llm = "llama3.2"
model_local = ChatOllama(model=ollama_llm)

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model_local
    | StrOutputParser()
)


In [89]:
# Question 1
chain.invoke("What went wrong with The Walking Dead?")

'According to the provided context, "What went wrong" refers to a specific issue mentioned multiple times:\n\nCasting is hard. \n\nSpecifically, it mentions that casting has always been tricky for the creators, and as they continue to increase the drama in their games, the bar for their actors keeps rising, making it increasingly difficult to find suitable talent.'

In [90]:
# Question 2
chain.invoke("Tell me about Telltale Games?")

'Based on the provided context, here\'s what I can tell you about Telltale Games:\n\n1. Unique approach: Telltale brings stories from TV and film into episodic "story" games.\n2. In-house operations: They handle licensing, design, development, publishing, and marketing all under one roof.\n3. Independent but not small: The company is independent but larger than typical "indie" studios.\n4. Collaborative decision-making: Product decisions are made together as a studio.\n5. Challenges with external partners: Working with external partners and understanding the concept of "playing a story" requires special attention.\n6. Focus on storytelling over technical aspects: They prioritize working out what it means to play a story rather than solely focusing on coding and technical challenges.\n\nThat\'s the main information I can gather from the provided context about Telltale Games!'

In [91]:
# Question 3
chain.invoke("What is Skybound?")

'The context only mentions "Skybound" in relation to partnering with them, implying that Skybound is a partner or collaborator of the company mentioned in the text. However, based on general knowledge, Skybound Entertainment is an American comic book publisher known for publishing titles such as The Walking Dead and other licensed properties.'

# Creacion GUI

In [92]:
def search_chroma(query, top_k):
    try:   
        results = vector_store.similarity_search(query, k=top_k)
        return "\n\n".join(
            [f"**Result {i+1}:**\n{doc.page_content}" for i, doc in enumerate(results)]
        )
    except Exception as e:
        return f"Error: {e}"

In [93]:
import gradio as gr

In [94]:
with gr.Blocks() as demo:
    gr.Markdown("### Chroma Database Search")
    
    with gr.Row():
        query_input = gr.Textbox(label="Enter Your Query", placeholder="Type your question here...")
        top_k_input = gr.Slider(1, 10, step=1, value=5, label="Number of Results")

    search_button = gr.Button("Search")
    output_box = gr.Textbox(label="Search Results", lines=15)

    search_button.click(fn=search_chroma, inputs=[query_input, top_k_input], outputs=output_box)

demo.launch()

* Running on local URL:  http://127.0.0.1:7867

To create a public link, set `share=True` in `launch()`.
